In [1]:
from Retrieval.retrieval import SparseRetrieval, DenseRetrieval,HybridRetrieval
from transformers import AutoTokenizer
from datasets import load_from_disk
import pickle

## Retrieval 클래스
- tokenizer, data_path, context_path를 인자로 가집니다.
- tokenizer -> 사용할 tokenizer를 사용합니다. DenseRetrieval의 경우 학습에 사용한 tokenizer를 사용해야합니다.
- context_path -> 가져올 corpus의 path를 인자로 넘겨주어야 합니다.
- data_path -> 가져온 문서나 embedding을 저장할 위치(미정)

# Retrieval 클래스의 method
- get_topk_doc_id_and_score : query에 대해서 top k개의 유사도가 높은 wiki id와 score를 가지고 옵니다.
- get_topk_doc_id_and_score_for_querys : query list 대해서 top k개의 유사도가 높은 wiki id와 score를 query - id list, query - score list 의 dict 형태로 전달해 줍니다.

In [2]:
valid_dataset = load_from_disk('/opt/ml/data/test_dataset/validation').to_pandas()
querys = valid_dataset['question']

## SparseRetrieval(elastic search) 예제

In [3]:
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base') # 토크나이저
elastic = SparseRetrieval(tokenizer=tokenizer) 
# 엘라스틱 서치는 nori 토크나이저를 사용하지만 exception이 생길 경우 bert tokenizer를 사용하는 
# BM25Okapi를 사용합니다.

Token indices sequence length is longer than the specified maximum sequence length for this model (1131 > 512). Running this sequence through the model will result in indexing errors
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [4]:
# get_topk_doc_id_and_score를 호출하면 해당 query에 대해 유사도가 높은 wiki 문서의 id와 그 점수를 top k 개 만큼 가져옵니다.
doc_ids, scores = elastic.get_topk_doc_id_and_score(querys[0],5)
print(doc_ids)
print(scores)

[43280, 47081, 35064, 24024, 42242]
[20.487246, 20.10372, 19.851677, 19.529999, 18.778242]


In [10]:
# context를 확인 하고 싶으면 클래스 변수인 wiki_id_context_dict을 사용하면 됩니다.
print(f'question : {querys[0]}')
for i in range(0,3):
    print(f'top_{i+1} passage score : {scores[i]}')
    print(elastic.wiki_id_context_dict[doc_ids[i]])
    print('-'*100)

question : 유령'은 어느 행성에서 지구로 왔는가?
top_1 passage : score : 20.487246
목성의 대기에서 보이는 줄무늬는 적도와 평행하면서 행성을 둘러싸는 대(zone)와 띠(belt)라고 불리는 물질의 반대 순환류에 의한 것이다. 대는 밝은 줄무늬로, 대기에서 상대적으로 고도가 높은 곳에 있다. 이들은 내부의 상승 기류를 가지고 있는 고기압 영역이다. 띠는 어두운 줄무늬로, 대기에서 상대적으로 고도가 낮은 곳에 있으며, 내부의 하강 기류를 가진다. 이들은 저기압 영역이다. 이러한 구조는 지구 대기의 고기압 및 저기압 세포와 어느정도 유사하나, 국지 작은 기압 세포와 상반되는 행성 전체를 둘러싸는 위도 줄무늬로서 매우 다른 구조를 가지고 있다. 이는 행성의 빠른 자전과 근본적인 대칭으로 인한 결과로 보인다. 행성에는 국지적인 가열을 일으키는 바다나 육지가 없으며 자전 속도는 지구보다 훨씬 빠르다. 행성에는 서로 다른 크기와 색상을 갖는 점과 같은 작은 구조들이 있다. 목성에서, 그러한 특색 중에서 가장 유명한 것은 대적점으로, 적어도 300년 동안 존재해 왔다. 이러한 구조의 실체는 거대한 폭풍이다. 그러한 점 중에 일부는 적란운이기도 하다.
----------------------------------------------------------------------------------------------------
top_2 passage : score : 20.10372
갈리프레이 (Gallifrey)는 영국의 SF 텔레비전 드라마 《닥터 후》에서 등장하는 행성이다. 드라마의 주인공인 닥터와 마스터를 비롯한 지금까지 등장한 모든 타임 로드의 고향이다. 카스터보로스 성단 내에서 "은하 중심에서 은하좌표로 10-0-11-0-0 하고도 0-2 지점"에 위치해 있으며 쌍성계를 이루고 있다. 닥터의 고향 행성은 드라마 방영 초반에는 밝혀지지 않다가, 2대 닥터 에피소드인 The War Games (1969)에서 닥터의 고향 행성이

In [27]:
# get_topk_doc_id_and_score_for_querys를 호출하고 인자로 query가 담긴 리스트와 top k를 인자로 넘겨주면
# query와 높은 유사도를 가진 wiki id 리스트와 score 리스트를 dict로 전달해 줍니다.
q_ids, q_scores = elastic.get_topk_doc_id_and_score_for_querys(querys.to_list(),10)

print(type(q_ids))
print(type(q_scores))
print(q_ids[querys[1]])
print(q_scores[querys[1]])

  0%|          | 0/600 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
100%|██████████| 600/600 [00:13<00:00, 45.56it/s]

<class 'dict'>
<class 'dict'>
[20081, 23179, 15556, 13590, 18470]
[23.784315, 21.420332, 21.085941, 20.766212, 19.601896]


In [28]:
print(f'question : {querys[1]}')
for i in range(0,3):
    print(f'top_{i+1} passage score : {q_scores[querys[1]][i]}')
    print(elastic.wiki_id_context_dict[q_ids[querys[1]][i]])
    print('-'*100)

question : 용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?
top_1 passage : score : 23.784315
SK에서 방출된 이후 그는 일본을 여행하며 잠시 신변을 정리하고 있었다. 그 때 좌완 선발 투수가 필요하여 니코스키에게 관심을 두고 있었던 두산 베어스가 SK 와이번스에 계약 양도를 신청하여 곧바로 맷 왓슨의 대체 용병으로 두산 베어스로 이적했고 두산 베어스의 선발진에 합류하여 비로소 첫 승을 따 냈으며 승리를 대부분 한화 이글스전에서 기록하여 한화 이글스에는 강력한 천적으로 자리매김했다. 그러나 예전부터 주로 원포인트 및 중간계투로 뛰다가 선발로 전환한 탓에 선발로는 아직 익숙하지 않아 3회까지 투구수가 70개 가까이 되는 경기가 대부분이라는 단점이 있었다. 2009년 9월 13일 KIA 타이거즈전에서 비로소 초반에 급격하게 늘어나는 투구수를 극복하여 승리를 따 내기도 했다. 두산 베어스 합류 후 12경기에 더 나서고 2009년 정규 시즌을 4승 8패, 평균 자책 3.78로 마무리했다. 완전한 선발 전업 후 후반기에 조금씩 나아지는 모습을 보였으나, 잠실야구장에서 롯데 자이언츠와 맞붙었던 2009년 9월 29일 준 플레이오프 1차전에서 무실점으로 3이닝을 호투하던 중 갑작스러운 어깨 통증으로 조기 강판당했고, 그 날 두산 베어스는 롯데 자이언츠에게 1차전을 내주었다. 진단 결과 극상근 손상 판정을 받아 포스트 시즌 전력에서 완전히 이탈하고 말았다. 플레이오프가 끝난 이후 이닝 소화 능력이 떨어진다는 이유로 두산 베어스가 재계약을 포기하면서 대한민국을 떠나게 되었다. 두산에서의 마지막이 좋지 않았지만, 그가 두산에 입단하여 2군에서 불펜 피칭을 했을 때 그를 지켜보았던 박종훈 당시 2군 감독은 니코스키를 두고 성격이 좋았다고 하였다.
----------------------------------------------------------------------------------------------------
top_2 p

## DenseRetrieval(Bert) 예제

In [36]:
# query encoder, context encoder를 학습시키고 사용한 모델의 tokenizer와 모델의 저장 위치를 인자로 가집니다.
dense_retrieval = DenseRetrieval(tokenizer, 'p_encoder/','q_encoder/')

Iteration: 100%|██████████| 1749/1749 [09:48<00:00,  2.97it/s]


In [37]:
# 문서를 가져오는 function은 Retrieval과 동일합니다. (SparseRetrieval 참고)
doc_ids, scores = dense_retrieval.get_topk_doc_id_and_score(querys[0],10)
print(doc_ids)
print(scores)

[55446, 44723, 25540, 17052, 21687, 40366, 41559, 49443, 49704, 55638]
[245.8260498046875, 244.90585327148438, 241.84640502929688, 239.195068359375, 237.49432373046875, 234.53616333007812, 234.44418334960938, 234.05784606933594, 233.98403930664062, 233.94419860839844]


In [38]:
print(f'question : {querys[0]}')
for i in range(0,3):
    print(f'top_{i+1} passage score : {scores[i]}')
    print(dense_retrieval.wiki_id_context_dict[doc_ids[i]])
    print('-'*100)

question : 유령'은 어느 행성에서 지구로 왔는가?
top_1 passage score : 245.8260498046875
공민왕 10년(1361년) 홍건적의 난을 진압하는 데에 안우 휘하에서 참전하여 세운 공으로 공민왕 12년(1363년) 2등 공신으로 책봉되었다. 공민왕 14년(1365년) 신돈의 참소로 유배되고 출가하여 산사로 들어갔던 찬성 이구수, 첨의평리 김귀, 판밀직 박춘을 배인길과 함께 바다에 빠뜨려 죽였으며, 이후 신돈이 실각하여 수원부로 유배되었을 때 이성림과 함께 신돈의 호송을 맡았다. 공민왕이 시해된 뒤 영녕군 왕유와 함께 이인임의 편에 서서 우왕을 옹립하는 데에 공을 세웠으며, 우왕 2년(1376년) 안주부원수(安州副元帥)로써 심왕(瀋王)이 훙서하였음을 보고하였고, 공민왕 시해 사건에 연루되어 유배된 자들의 친족들에 대해 처형하지 말 것을 진언하였다. 우왕 3년(1377년) 2월 양광도도원수가 되었다. 5월에 왜적이 여미현(餘美縣)을 공격하자 이를 치러 나서서 산을 타고 면주 방면으로 달아나는 왜구를 추격해 왜구 한 명을 죽였으나, 왜구는 이어 가야사를 침입하였고 우왕은 체복사(體覆使) 최인철(崔仁哲)을 보내어 왜구를 잡지 못한 것을 책망하였다. 왜적은 강화도에서부터 양광도에 이르는 바닷가 고을들을 차례로 약탈하면서 경양(慶陽)을 치고 안성군(安城郡)에 이르렀으며, 그 사이 고려군의 전함을 빼앗아 22척에서 50척으로 선단의 규모가 늘어난 것은 물론, 빼앗은 고려군 전함을 앞세워 고려의 군민이 방심하게 해 놓고 기습하거나 군사를 거느리고 안성을 구원하러 왔던 수원부사(水原府使) 박승직(朴承直)을 왜적이 이미 물러간 것처럼 속이고 유인하여 그의 군세를 궤멸시키는 등 고려군을 대상으로 기만전술을 펼쳤다. 왕안덕은 왜구의 군세를 보고 겁을 내어 나아가지 못하고 부원수(副元帥) 인해(印海)와 양천원수(陽川元帥) 홍인계(洪仁桂)를 불러 후퇴시켜 가천역(加川驛)에 진을 치게 한 다음 돌아가는 적을 요격하려고 하였으나, 이를 간파한 왜구는 왕안

In [39]:
q_ids, q_scores = dense_retrieval.get_topk_doc_id_and_score_for_querys(querys.to_list(),10)

print(type(q_ids))
print(type(q_scores))
print(q_ids[querys[1]])
print(q_scores[querys[1]])

100%|██████████| 600/600 [00:00<00:00, 1326.12it/s]

<class 'dict'>
<class 'dict'>
[60093, 49304, 30111, 21620, 54027, 26256, 54288, 35142, 40232, 57448]
[262.4776916503906, 261.6727600097656, 257.96929931640625, 255.0395050048828, 252.2115020751953, 251.95045471191406, 251.12380981445312, 250.7698211669922, 250.66644287109375, 250.35877990722656]


In [40]:
print(f'question : {querys[1]}')
for i in range(0,3):
    print(f'top_{i+1} passage score : {q_scores[querys[1]][i]}')
    print(dense_retrieval.wiki_id_context_dict[q_ids[querys[1]][i]])
    print('-'*100)

question : 용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?
top_1 passage score : 262.4776916503906
신수영(愼守英, ? ~ 1506년 9월 2일)은 조선 전기의 문신이자 외척이다. 본관은 거창(居昌)으로, 아버지는 영의정 신승선이고, 연산군의 처남이며, 세종대왕의 5남 임영대군 구의 외손이다. 세종, 예종, 연산군, 중종 4대의 외척이 된다. 연산군 때 도감낭청을 시작으로 1500년(연산군 6년) 의빈부경력에서 5품이 되었다가 그해에 동부승지로 발탁되어 우승지, 우부승지, 좌부승지, 우승지를 거쳐 도승지로 발탁되었다. 이후 호조참판, 형조참판, 병조참판을 거쳐 1506년 4월 형조판서 겸 오위도총부도총관이 되었다가 그해 9월의 중종반정으로 타살당했다.
----------------------------------------------------------------------------------------------------
top_2 passage score : 261.6727600097656
까치상어에 대한 첫 과학적 기술은 일본의 건조 표본(dried specimen)을 기반으로 한 1838~41년 책 "Systematische Beschreibung der Plagiostomen"에서 독일의 생물학자 요하네스 페터 뮐러와 Friedrich Gustav Jakob Henle에 의해 저술되었다. 종소명의 경우 돔발 상어를 뜻하는 고대 그리스어 skylion이 어원인 scyllium으로 정했으며, 속은 Triakis로 분류했다 해당 속 내에서 레오파드 상어(T. (Triakis) semifasciata)와 함께 아속 Triakis에 위치해 있다
----------------------------------------------------------------------------------------------------
top_3 passage score : 257.96929931640625
《생방송 세븐

## HybridRetrieval(Dense_retrieval + Sparse_retrieval) 예제

In [41]:
# Dense_retrieval을 학습시켰던 tokenizer와 p_encdoer, q_encoder의 path를 인자로 받습니다.
hybrid_retrieval = HybridRetrieval(tokenizer, 'p_encoder/','q_encoder/')

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [42]:
doc_ids, scores = hybrid_retrieval.get_topk_doc_id_and_score(querys[0],10)

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [46]:
print(f'question : {querys[0]}')
for i in range(0,3):
    print(f'top_{i+1} passage score : {scores[i]}')
    print(hybrid_retrieval.wiki_id_context_dict[doc_ids[i]])
    print('-'*100)

question : 유령'은 어느 행성에서 지구로 왔는가?
top_1 passage score : 235.19189475195313
더크 젠틀리의 성스러운 탐정사무소의 줄거리는 이야기의 중추적인 부위에 자리잡은 시간 여행이란 주제 때문에 줄거리가 이어져 있지 않다. 40억년전 지구에서, 사락사라의 사람들이 자신들만의 낙원을 만들기 위해 지구에 착륙하였다. 그러나, 기술자의 게으름으로 인해서 착륙선은 폭발하였으며, 안에 타고 있던 사락사라 사람들은 모두다 죽어버렸으며 그 폭발에 의한 에너지가 아미노산을 만들어 지구에 생명이 생기게 되었다. 그 게으른 엔지니어는 유령이 되어 그가 저지른 잘못을 돌리기 위해 돌아다니면서, 인류의 발전에 영향을 미쳐왔다. 1800년 초에 유령은 케임브리지 세인트 체드 단과대학의 리즈 교수가 타임머신을 가졌다는 걸 알게 되고, 리즈교수를 손에 넣을려고 하였으나 실패하고 만다. 그리고 유령은 새뮤얼 테일러 콜리지에게 붙어, 그의 시 쿠빌라이 칸과 늙은 뱃사람의 노래에 착륙선을 고칠방법을 적게 만든다. 유령은 200여 년 동안 콜리지의 작품에 감명을 받은 영혼을 찾아다녔으며, 21세기무렵 마이클 웬튼 윅스라는 전 예술잡지 편집장을 찾게 된다. 유령은 마이클이 콜리지의 작품을 읽게 만들었으며, 그를 조종하기 위해 마이클로 하여금 마이클의 후임 편집장인 알버트 로스를 살해하게 된다. 유령의 다음 행동은 다른 사람에게 영향을 끼칠 숙주를 찾는 일이었다. 리즈는 어린 소녀를 재미있게 해주기 위해 타임머신을 가동하였으며, 유령이 사용하기위해 데려온 전자 수도사는 웨이포인트 테크놀러지 II의 사장인 고든 웨이를 죽여버린다. 리즈의 학생이었으며, 웨이포인트 테크놀러지 II의 컴퓨터 프로그래머인 리처드 맥더프는 그의 여자친구인 수잔 웨이의 집의 자동응답기에 저장된 메시지를 지우기 위해서 몰래 집에 침입하게 되는데, 그때 맥더프는 알지 못하게 유령에게 이용되었다. 이건 만물의 상호 연관성을 믿는 맥더프의 친구인 더크 젠틀리를 끌여들이게 되었다. 더크 젠틀

In [44]:
q_ids, q_scores = hybrid_retrieval.get_topk_doc_id_and_score_for_querys(querys.to_list(),10)

print(type(q_ids))
print(type(q_scores))
print(q_ids[querys[0]])
print(q_scores[querys[0]])

100%|██████████| 600/600 [00:38<00:00, 15.70it/s]

<class 'dict'>
<class 'dict'>
[24024, 9781, 35064, 47081, 10509, 42242, 34043, 29886, 19993, 43280]
[235.19189475195313, 224.04907226171875, 218.28617101855468, 216.62479238769532, 215.91527337402343, 215.55450237597657, 215.10580463867188, 210.36959424707032, 203.38334273242188, 198.99333425683594]


In [47]:
print(f'question : {querys[1]}')
for i in range(0,3):
    print(f'top_{i+1} passage score : {q_scores[querys[1]][i]}')
    print(elastic.wiki_id_context_dict[q_ids[querys[1]][i]])
    print('-'*100)

question : 돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?
top_1 passage score : 288.7824329941406
1928년 경상북도 성주에서 출생하여 지난날 한때 경상북도 대구에서 잠시 유아기를 보낸 적이 있는 이용수는 유모로 일하는 어머니 대신 동생을 돌보며 면사공장에 다니다가 16세이던 1944년 군 위안부로 타이완(대만)에 끌려갔다가 1946년 고향으로 돌아왔다. 1994년 5월 나가노 시게토 법무상의 태평양전은 침략 전쟁이 아니고 ‘위안부’는 공창이었다는 발언에 항의하기 위해 일본을 방문한 것을 계기로 이후 100회 이상 열린 증언 모임에 초청되어 증언했다 2000년 도쿄에서 개최된 여성국제전범법정에서 제2차 세계 대전 중 경험을 증언했고, 미국 홀로코스트 기념관에서도 증언했다. 2007년에 미국 의회에서 다른 2명의 위안부 피해자 (김군자, 얀 루프 오헤른)과 공동으로 자신의 피해경험을 증언했다. 이용수 등 피해자들의 증언은 미국 하원에서 일본 정부가 위안부에 공식적으로 사과할 것을 요구하는 결의안을 통과시키는데 결정적인 역할을 했고 결의안은 만장일치로 통과되었다. 영화 아이 캔 스피크는 이 역사적인 사건을 모티브로 했다. 2015년 대한민국 국회 의원회관에서 일본군 위안부 피해자 할머니들의 임상미술치료 작품을 전시하는 ‘역사가 된 그림’ 전시회에 참석했다. 아베 신조 일본 총리의 사죄를 촉구하면서 그의 방미 동선에 맞춰 보스턴과 워싱턴DC 등에서 항위 시위를 벌였다. 2015년 한·일 일본군 위안부 협상 타결에 반대하였다. 2016년 유엔 본부에서 세계 언론을 대상으로 기자회견을 갖고 위안부 피해의 실상을 증언했다. 2015년 미국 샌프란시스코 차이나타운 인근 세인트 메리 파크에 한ㅡ중 ㅡ필 위안부 기념물을 세우기 위한 공청회에 유일한 위안부 피해자 증인으로 참석해서 일본 정부의 후원을 받는 반대파를 꺾는 핵심역할을 했다. 이 소녀상 실제 건립은 2017년으로서 박근혜 정부 한일합의 이후였는데,일본 